In [42]:
!pip install kiwipiepy

In [72]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, InputLayer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import re
from kiwipiepy import Kiwi

#1.Seq2Seq 모델을 활용한 챗봇


##1-1.데이터 로드 및 확인
- 한글 자연어처리 데이터 셋인 Korpora 중 챗봇용 데이터 셋인 KoreanChatbotKorpus를 사용
- 데이터 출처 : https://github.com/songys/Chatbot_data
- 챗봇 트레이닝용 문답 페어 : 11,823
- 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링

In [44]:
corpus=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RNN/data/ChatbotData.csv')
corpus.shape

(11823, 3)

In [45]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [46]:
# 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링
corpus['label'].value_counts()

,count
label,
0,5290
1,3570
2,2963


In [47]:
# 질의 Q와 답변 A로 구성 >> 데이터를 리스트 형태로 저장

df_Q=corpus['Q']
texts=df_Q.values.tolist()

df_A=corpus['A']
pairs=df_A.values.tolist()

list(zip(texts, pairs))[:5]

[('12시 땡!', '하루가 또 가네요.'),
 ('1지망 학교 떨어졌어', '위로해 드립니다.'),
 ('3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('PPL 심하네', '눈살이 찌푸려지죠.')]

##1-2.데이터 전처리

In [48]:
# 1) 데이터 정제 >> 특수문자 제거
def clean_sentence(sentence):
    sentence=re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence

In [49]:
clean_sentence('12시 땡^^!??')

'12시 땡'

In [50]:
clean_sentence('abcef가나다^^$%@12시 땡^^!??')

'가나다12시 땡'

In [51]:
# 2) 한글 형태소 분석기
kiwi=Kiwi()
def process_morph(sentence):
    tokens=kiwi.tokenize(sentence)
    morphs=[token.form for token in tokens]
    return ' '.join(morphs)   # 추출된 형태소들을 공백(' ')으로 이어붙여 하나의 문자열

##1-3. Seq2Seq모델에 사용되는 3가지 데이터셋을 구성
- question : encoder input 데이터셋(질의)
- answer_input : decoder input 데이터셋, SOS 토큰을 문장 처음에 추가
- answer_output : decoder output 데이터셋, EOS 토큰을 문장 마지막에 추가

In [52]:
# 3) 데이터셋 구성
def clean_and_morph(sentence, is_question=True):  # 문장, 질문답변
    sentence=clean_sentence(sentence)    #  1) 데이터 정제
    sentence=process_morph(sentence)     #  2) 형태소 변환

    if is_question:  # 질문
        return sentence
    else:  #  답변은 시작/끝 토큰을 붙여야 디코더 학습시 유용
        return ('<SOS> ' + sentence, sentence + ' <EOS>')    # 공백 Check

In [53]:
# 4) 데이터셋 생성
def preprocess(texts, pairs):
    questions=[]
    answer_in=[]
    answer_out=[]

    # 질의에 대한 전처리
    for text in texts:
        # 전처리와 morph 수행
        question=clean_and_morph(text, is_question=True)  # 3) 호출 >> 1), 2) 호출
        questions.append(question)

    # 답변에 대한 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)

    return questions, answer_in, answer_out

In [54]:
questions, answer_in, answer_out=preprocess(texts, pairs)   # 4) 호출
questions[:5]

['12 시 땡',
 '1 지망 학교 떨어지 었 어',
 '3 박 4 일 놀 러 가 고 싶 다',
 '3 박 4 일 정도 놀 러 가 고 싶 다',
 '심 하 네']

In [55]:
answer_in[:5]

['<SOS> 하루 가 또 가 네요',
 '<SOS> 위로 하 어 드리 ᆸ니다',
 '<SOS> 여행 은 언제나 좋 죠',
 '<SOS> 여행 은 언제나 좋 죠',
 '<SOS> 눈살 이 찌푸리 어 지 죠']

In [56]:
answer_out[:5]

['하루 가 또 가 네요 <EOS>',
 '위로 하 어 드리 ᆸ니다 <EOS>',
 '여행 은 언제나 좋 죠 <EOS>',
 '여행 은 언제나 좋 죠 <EOS>',
 '눈살 이 찌푸리 어 지 죠 <EOS>']

##1-4.토큰화

In [61]:
# len(texts)   # 11823개의 데이터 전부를 학습하기에는 램이 부족하기 때문에 데이터 수를 조절
train_texts=texts[:5000]
train_pairs=pairs[:5000]

questions, answer_in, answer_out=preprocess(train_texts, train_pairs)   # 4) 호출
len(questions), len(answer_in), len(answer_out)

(5000, 5000, 5000)

In [62]:
# 토큰화를 위해 모든 문장을 합쳐줌
all_sentences=questions + answer_in + answer_out
a=(' '.join(questions) + ' '.join(answer_in) + ' '.join(answer_out)).split()
len(a), len(set(a))

(120016, 3352)

In [63]:
# 토큰의 정의
# shift + tab 확인 >> 아무 문자도 제거하지 않음, 대소문자를 구분
tokenizer=Tokenizer(filters='', lower=False, oov_token='<OOV>')

# 토근 문장에 대한 Word-Index Vocabulary(단어 사전)을 만듭니다.
tokenizer.fit_on_texts(all_sentences)

In [64]:
for word, idx in tokenizer.word_index.items():   # 빈도수 기반으로 idx값 부여
    print(f'{word}\t\t => \t{idx}')
    if idx > 10:
        break

<OOV>		 => 	1
하		 => 	2
어		 => 	3
이		 => 	4
<SOS>		 => 	5
<EOS>		 => 	6
어요		 => 	7
보		 => 	8
는		 => 	9
세요		 => 	10
ᆯ		 => 	11


In [65]:
print(len(tokenizer.word_index))

3351


In [66]:
# 텍스트를 시퀀스로 인코딩
question_seq=tokenizer.texts_to_sequences(questions)
answer_in_seq=tokenizer.texts_to_sequences(answer_in)
answer_out_seq=tokenizer.texts_to_sequences(answer_out)
# len(question_seq)

In [67]:
# 2) 문장의 길이 맞추기
MAX_LENGTH=30

# 문장이 30을 넘어갈 경우 뒤에서 자르기, 뒤에서부터 0으로 패딩
question_pad=pad_sequences(question_seq, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_in_pad=pad_sequences(answer_in_seq, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_out_pad=pad_sequences(answer_out_seq, maxlen=MAX_LENGTH, truncating='post', padding='post')

question_pad.shape, answer_in_pad.shape, answer_out_pad.shape

((5000, 30), (5000, 30), (5000, 30))

#2.모델 생성

##2-1.Encoder & Decoder & Seq2Seq
- 추후 : 추론, 정수형 인코딩 텍스트 변환 등 확장성

In [77]:
class Encoder(tf.keras.Model):
    # 객체 생성 시 (한번) >> 초기화
    def __init__(self, units, vocab_size, embedding_dim, max_len):
        super(Encoder, self).__init__()
        self.input_layer=InputLayer(shape=(max_len,))
        self.embedding=Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.dropout=Dropout(0.2)
        self.lstm=LSTM(units, return_state=True,  return_sequences=True)

    # 모델 호출 시 (매번) >> 순전파 로직
    def call(self, inputs):
        x=self.embedding(inputs)
        x=self.dropout(x)
        x, hidden_state, cell_state=self.lstm(x)
        return [hidden_state, cell_state]

In [78]:
class Decoder(tf.keras.Model):
    # 객체 생성 시 (한번) >> 초기화
    def __init__(self, units, vocab_size, embedding_dim, max_len):
        super(Decoder, self).__init__()
        self.input_layer=InputLayer(shape=(max_len,))
        self.embedding=Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.dropout=Dropout(0.2)
        self.lstm=LSTM(units, return_state=True,  return_sequences=True)
        self.dense=Dense(vocab_size, activation='softmax')

   # 모델 호출 시 (매번) >> 순전파 로직
    def call(self, inputs, initial_state):
        x=self.embedding(inputs)
        x=self.dropout(x)
        x, hidden_state, cell_state=self.lstm(x, initial_state=initial_state)
        x=self.dense(x)
        return x, hidden_state, cell_state

In [79]:
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, max_len, start_token, end_token):  # (128, 3364, 100, 30, 5, 6)
        super(Seq2Seq, self).__init__()
        self.start_token=start_token
        self.end_token=end_token
        self.time_steps=max_len

        self.encoder=Encoder(units, vocab_size, embedding_dim, max_len)
        self.decoder=Decoder(units, vocab_size, embedding_dim, max_len)

    def call(self, inputs):
        encoder_inputs, decoder_inputs=inputs         # 질문문장, 정답문장
        context_vector=self.encoder(encoder_inputs)   # 문맥벡터를 얻는다. (h, c) >> 디코더의 초기상태
        decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs, initial_state=context_vector)  # 각 시점에서의 softmax 예측값, h, c
        return decoder_outputs

In [80]:
# 모델 학습
seq2seq=Seq2Seq(units=128,
                vocab_size=len(tokenizer.word_index)+1,
                embedding_dim=100,
                max_len=MAX_LENGTH,
                start_token=tokenizer.word_index['<SOS>'],
                end_token=tokenizer.word_index['<EOS>'])

seq2seq.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [81]:
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=5)

In [82]:
# 모델 학습
seq2seq.fit([question_pad, answer_in_pad], answer_out_pad, batch_size=64,
            epochs=20, validation_split=0.2, callbacks=[early_stopping_callback])

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 44s 605ms/step - accuracy: 0.6585 - loss: 5.4163 - val_accuracy: 0.7012 - val_loss: 1.7516
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 39s 573ms/step - accuracy: 0.7065 - loss: 1.6448 - val_accuracy: 0.7355 - val_loss: 1.5583
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 44s 618ms/step - accuracy: 0.7464 - loss: 1.4703 - val_accuracy: 0.7445 - val_loss: 1.4860
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 606ms/step - accuracy: 0.7499 - loss: 1.4135 - val_accuracy: 0.7473 - val_loss: 1.4317
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 616ms/step - accuracy: 0.7572 - loss: 1.3357 - val_accuracy: 0.7588 - val_loss: 1.3832
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 39s 584ms/step - accuracy: 0.7676 - loss: 1.3029 - val_accuracy: 0.7751 - val_loss: 1.3343
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 590ms/step - accuracy: 0.7808 - loss: 1.2392 - val_accuracy: 0.7848 - val_loss: 1.2856
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 585ms/step - accuracy: 0.7910 - loss: 1.1929 - val_accu

In [83]:
test_texts=texts[5000:6000]
test_pairs=pairs[5000:6000]
test_questions, test_answer_in, test_answer_out=preprocess(test_texts, test_pairs)   # 정제
print(len(test_questions), len(test_answer_in), len(test_answer_out))

test_all_sentences=questions + answer_in + answer_out
test_a=(' '.join(questions) + ' '.join(answer_in) + ' '.join(answer_out)).split()
len(test_a), len(set(test_a))

tokenizer=Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(all_sentences)
print(len(tokenizer.word_index))          # 토큰

test_question_seq=tokenizer.texts_to_sequences(questions)
test_answer_in_seq=tokenizer.texts_to_sequences(answer_in)
test_answer_out_seq=tokenizer.texts_to_sequences(answer_out)
print(len(test_question_seq))

1000 1000 1000
3351
5000


In [84]:
# 패딩
test_question_pad=pad_sequences(question_seq, maxlen=MAX_LENGTH, truncating='post', padding='post')
test_answer_in_pad=pad_sequences(answer_in_seq, maxlen=MAX_LENGTH, truncating='post', padding='post')
test_answer_out_pad=pad_sequences(answer_out_seq, maxlen=MAX_LENGTH, truncating='post', padding='post')
test_question_pad.shape, test_answer_in_pad.shape, test_answer_out_pad.shape

((5000, 30), (5000, 30), (5000, 30))

In [85]:
loss, acc=seq2seq.evaluate([test_question_pad, test_answer_in_pad], test_answer_out_pad)
print(f'Loss: {loss:.4f}, Accuracy: {acc:.4f}')

157/157 ━━━━━━━━━━━━━━━━━━━━ 18s 115ms/step - accuracy: 0.8393 - loss: 0.8537
Loss: 0.8858, Accuracy: 0.8374
